In [1]:
import os
import pickle
from sparse_to_array import *

In [2]:
datadir = 'data/vectors/'
datafile = '20_newsgroups_simple.p'
os.chdir('..')
os.chdir(datadir)
os.listdir()

['20_newsgroups_complex.p', '20_newsgroups_simple.p']

In [3]:
with open(datafile,'rb') as infile:
    corpus = pickle.load(infile)

vocab = corpus[0]
docs = corpus[1]

In [4]:
docvecs,vocab = sparse_docs_to_array(docs,vocab)